# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [52]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [53]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/workspace/home


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [54]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
print(full_data_rows_list[0])

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


8056
['', 'Logged In', 'Adler', 'M', '0', 'Barrera', '', 'free', 'New York-Newark-Jersey City, NY-NJ-PA', 'GET', 'Home', '1.54084E+12', '248', '', '200', '1.54147E+12', '100']


In [55]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [56]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [57]:
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS udacity 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print(e)

#### Set Keyspace

In [58]:
try:
    session.set_keyspace('udacity')
except Exception as e:
    print(e)

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




In [300]:
# Save the column types and the respective index in the .csv.
# This data frame will help to generalize the creation, insert and validation of each table.

df_cols = pd.DataFrame({'artist':        ['text',  0],
                        'song':          ['text',  9], 
                        'length':        ['float', 5], 
                        'sessionid':     ['int',   8], 
                        'iteminsession': ['int',   3],
                        'firstname':     ['text',  1], 
                        'lastname':      ['text',  4],  
                        'userid':        ['int',   10]}).T

df_cols.columns = ['dtype', 'line_index']

In [299]:
def parse_cols(df, include_type=False):
    """ Parse the columns.
    
    
    Args:
        df (pd.DataFrame)  : pandas dataframe whose index is the column name. It has a dtype column with the column type.
        include_type (bool): bool to decide whether to include the column type in the concatenation. Default False.
        
    Return:
        string of column names concatenated with a comma (can also include the column type).
        
    Example:
    >>> parse_cols({'artist': 'text', 'length': 'float', include_type=True)
    'artist text, length float'
    """
    
    sep = ', '
    
    if include_type:
        parsed_cols = sep.join([df.index[i] + ' ' + df['dtype'][i] for i in range(len(df))])
    else:
        parsed_cols = sep.join([df.index[i]                        for i in range(len(df))])
    
    return parsed_cols

In [297]:
def get_line_values(df,lines):
    """Return the line value converted to the correct data type.
    
    Args:
        df (pd.DataFrame)  : pandas dataframe whose index is the column name. It has a dtype column with the column type.
        lines (list): row values (read from .csv).
        
    Return:
        tuple with converted values.
    """
    funs = {'text': str, 'float': float, 'int': int}
    line_values = []
    for i in range(len(df)):
        dtype = df['dtype'][i]
        line_index = df['line_index'][i]
        line_value = funs[dtype](lines[line_index])
        line_values.append(line_value)
    return tuple(line_values)



In [303]:
def create_table(tbl,cols_create,cols_pk):
    """Create a table in apache cassandra.
    """
    
    cols_pk_parsed     = parse_cols(df_cols.loc[cols_pk],    include_type=False)
    cols_create_parsed = parse_cols(df_cols.loc[cols_create],include_type=True)
    
    query = f"CREATE TABLE IF NOT EXISTS {tbl} "
    query = query + f"({cols_create_parsed}, PRIMARY KEY ({cols_pk_parsed}))"

    try:
        session.execute(query)
    except Exception as e:
        print(e)         

In [304]:
def insert_data(tbl, df_cols, cols_create):
    """Insert data into a table in apache cassandra.
    """
    
    file = 'event_datafile_new.csv'

    cols_insert_parsed = parse_cols(df_cols.loc[cols_create],include_type=False)
    insert_placeholder = ', '.join(['%s'] * len(cols_create))
    
    with open(file, encoding = 'utf8') as f:
        csvreader = csv.reader(f)
        next(csvreader) # skip header
        for line in csvreader:
            line_values = get_line_values(df_cols.loc[cols_create],line)
            query = f"INSERT INTO {tbl} ({cols_insert_parsed})"
            query = query + f" VALUES ({insert_placeholder})"
            session.execute(query, line_values)

In [340]:
from prettytable import PrettyTable

def verify_data(tbl, df_cols, cols_select, where_statement):
    """Prints the table rows to verify the content.
    """
    
    cols_select_parsed = parse_cols(df_cols.loc[cols_select],include_type=False)
    
    query = f"select {cols_select_parsed} from {tbl} where {where_statement}"
    
    try:
        rows = session.execute(query)
    except Exception as e:
        print(e)
        
    t = PrettyTable(cols_select)
    for row in rows:
        row_content = []
        for i,attr in enumerate(cols_select):
            row_content.append(row[i])
        t.add_row(row_content)
    print(t)

___

> 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


To perform this query we need the information about the sessions and the songs listened, so the table that will be build will be composed of the following columns:
- Partition key: `sessionid`
- Clustering columns: `iteminsession`
- Selected columns: `artist`, `song`, `length`


In [348]:
# Query 1: Give me the artist, song title and song's length in the music app history that was heard 
# during sessionId = 338, and itemInSession = 4

tbl_query_1     = 'tbl_song_details'

cols_partkey    = ['sessionid']
cols_cluster    = ['iteminsession']
cols_select     = ['artist', 'song', 'length']
cols_primkey    = cols_partkey + cols_cluster
cols_create     = cols_primkey + cols_select
# cols_create     = ['artist', 'song', 'length', 'sessionid', 'iteminsession']

where_statement = 'sessionid=338 and iteminsession=4'


In [349]:
create_table(tbl_query_1, cols_create, cols_primkey)

In [350]:
insert_data(tbl_query_1, df_cols, cols_create)

In [351]:
verify_data(tbl_query_1, df_cols, cols_select, where_statement)

+-----------+---------------------------------+--------------------+
|   artist  |               song              |       length       |
+-----------+---------------------------------+--------------------+
| Faithless | Music Matters (Mark Knight Dub) | 495.30731201171875 |
+-----------+---------------------------------+--------------------+


___

> 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182.


To perform this query we need the information about the sessions, songs and users, so the table that will be build will be composed of the following columns:
- Partition key: `userid`, `sessionid`
- Clustering columns: `iteminsession`
- Selected columns: `artist`, `song`, `firstname`, `lastname`


In [352]:
# Query 2: Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name)
# for userid = 10, sessionid = 182

tbl_query_2     = 'tbl_artist_details'

cols_partkey    = ['userid', 'sessionid']
cols_cluster    = ['iteminsession']
cols_select     = ['artist', 'song', 'firstname', 'lastname']
cols_primkey    = cols_partkey + cols_cluster
cols_create     = cols_primkey + cols_select

where_statement = 'userid = 10 and sessionid = 182'


In [353]:
create_table(tbl_query_2, cols_create, cols_primkey)

In [354]:
insert_data(tbl_query_2, df_cols, cols_create)

In [355]:
verify_data(tbl_query_2, df_cols, cols_select, where_statement)

+-------------------+------------------------------------------------------+-----------+----------+
|       artist      |                         song                         | firstname | lastname |
+-------------------+------------------------------------------------------+-----------+----------+
|  Down To The Bone |                  Keep On Keepin' On                  |   Sylvie  |   Cruz   |
|    Three Drives   |                     Greece 2000                      |   Sylvie  |   Cruz   |
| Sebastien Tellier |                      Kilometer                       |   Sylvie  |   Cruz   |
|   Lonnie Gordon   | Catch You Baby (Steve Pitron & Max Sanna Radio Edit) |   Sylvie  |   Cruz   |
+-------------------+------------------------------------------------------+-----------+----------+


___

> 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'


To perform this query we need the information about the songs and users, so the table that will be build will be composed of the following columns:
- Partition key: `song`
- Clustering columns: `userid`
- Selected columns: `firstname`, `lastname`


In [360]:
# Query 3: Give me every user name (first and last) in my music app history who listened to 
# the song 'All Hands Against His Own'

tbl_query_3     = 'tbl_user_details'

cols_partkey    = ['song']
cols_cluster    = ['userid']
cols_select     = ['firstname', 'lastname']
cols_primkey    = cols_partkey + cols_cluster
cols_create     = cols_primkey + cols_select

where_statement = "song = 'All Hands Against His Own'"

In [361]:
create_table(tbl_query_3, cols_create, cols_primkey)

In [362]:
insert_data(tbl_query_3, df_cols, cols_create)

In [363]:
verify_data(tbl_query_3, df_cols, cols_select, where_statement)

+------------+----------+
| firstname  | lastname |
+------------+----------+
| Jacqueline |  Lynch   |
|   Tegan    |  Levine  |
|    Sara    | Johnson  |
+------------+----------+


___

### Drop the tables before closing out the sessions

In [345]:
tbls = [tbl_query_1, tbl_query_2, tbl_query_3]

for tbl in tbls:
    query = f"drop table {tbl}"
    try:
        rows = session.execute(query)
    except Exception as e:
        print(e)

### Close the session and cluster connection¶

In [50]:
session.shutdown()
cluster.shutdown()